In [2]:
import collections
import itertools
import random
import time
import _thread as thread
import threading

from graph_nets import graphs
from graph_nets import utils_np
from graph_nets import utils_tf
from graph_nets.demos import models
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from copy import deepcopy
from math import ceil, floor, sqrt
from scipy import spatial
from string import ascii_lowercase, ascii_uppercase
import tensorflow as tf

SEED = 1
random.seed(SEED)
np.random.seed(SEED)
tf.set_random_seed(SEED)

In [3]:
# Implements Donald Knuth's "Algorithm X"
# http://lanl.arxiv.org/pdf/cs/0011047.pdf
# Based on implementation by Gareth Rees
# https://codereview.stackexchange.com/questions/37415/sudoku-using-exact-cover-solver

DIGITS = '123456789' + ascii_uppercase + ascii_lowercase


def make_grid(n):
    """Return a Sudoku grid of size n x n."""
    return [[-1] * n for _ in range(n)]

def puzzle_to_grid(puzzle):
    """Convert printed representation of a Sudoku puzzle into grid
    representation (a list of lists of numbers, or -1 if unknown).

    """
    puzzle = puzzle.split()
    grid = make_grid(len(puzzle))
    for y, row in enumerate(puzzle):
        for x, d in enumerate(row):
            grid[y][x] = DIGITS.find(d)
    return grid

def grid_to_puzzle(grid):
    """Convert grid representation of a Sudoku puzzle (a list of lists of
    numbers, or -1 if unknown) into printed representation.

    """
    return '\n'.join(''.join('.' if d == -1 else DIGITS[d] for d in row)
                     for row in grid)


class ExactCover(collections.Iterator):
    """An iterator that yields solutions to an EXACT COVER problem.

    An EXACT COVER problem consists of a set of "choices". Each choice
    satisfies one or more "constraints". Choices and constraints may
    be represented by any hashable Python objects, with the proviso
    that all choices must be distinct, as must all constraints.

    A solution is a list of choices such that each constraint is
    satisfied by exactly one of the choices in the solution.

    The constructor takes three arguments:

    constraints  A map from each choice to an iterable of constraints
                 satisfied by that choice.
    initial      An iterable of choices that must appear in every
                 solution. (Default: no choices.)
    random       Generate solutions in random order? (Default: False.)

    For example:

        >>> next(ExactCover(dict(A = [1, 4, 7],
        ...                      B = [1, 4],
        ...                      C = [4, 5, 7],
        ...                      D = [3, 5, 6],
        ...                      E = [2, 3, 6, 7],
        ...                      F = [2, 7])))
        ['B', 'D', 'F']

    """

    def __init__(self, constraints, initial=(), random=False):
        self.random = random
        self.constraints = constraints

        # A map from constraint to the set of choices that satisfy
        # that constraint.
        self.choices = collections.defaultdict(set)
        for i in self.constraints:
            for j in self.constraints[i]:
                self.choices[j].add(i)

        # The set of constraints which are currently unsatisfied.
        self.unsatisfied = set(self.choices)

        # The partial solution currently under consideration,
        # implemented as a stack of choices.
        self.solution = []

        # Make all the initial choices.
        try:
            for i in initial:
                self._choose(i)
            self.it = self._solve()
        except KeyError:
            # Initial choices were contradictory, so there are no solutions.
            self.it = iter(())

    def __next__(self):
        return next(self.it)

    next = __next__             # for compatibility with Python 2

    def _solve(self):
        if not self.unsatisfied:
            # No remaining unsatisfied constraints.
            yield list(self.solution)
            return

        # Find the constraint with the fewest remaining choices.
        best = min(self.unsatisfied, key=lambda j:len(self.choices[j]))
        choices = list(self.choices[best])
        if self.random:
            random.shuffle(choices)

        # Try each choice in turn and recurse.
        for i in choices:
            self._choose(i)
            for solution in self._solve():
                yield solution
            self._unchoose(i)

    def _choose(self, i):
        """Make choice i; mark constraints satisfied; and remove any
        choices that clash with it.

        """
        self.solution.append(i)
        for j in self.constraints[i]:
            self.unsatisfied.remove(j)
            for k in self.choices[j]:
                for l in self.constraints[k]:
                    if l != j:
                        self.choices[l].remove(k)

    def _unchoose(self, i):
        """Unmake choice i; restore constraints and choices."""
        self.solution.pop()
        for j in self.constraints[i]:
            self.unsatisfied.add(j)
            for k in self.choices[j]:
                for l in self.constraints[k]:
                    if l != j:
                        self.choices[l].add(k)


class Sudoku(collections.Iterator):
    """An iterator that yields the solutions to a Sudoku problem.

    The constructor takes three arguments:

    puzzle   The puzzle to solve, in the form of a string of n
             words, each word consisting of n characters, either a
             digit or a dot indicating a blank square.
             (Default: the blank puzzle.)
    n        The size of the puzzle. (Default: 9.)
    m        The width of the blocks. (Default: the square root of n,
             rounded up.)
    random   Generate solutions in random order? (Default: False.)

    For example:

        >>> print(next(Sudoku('''...84...9
        ...                      ..1.....5
        ...                      8...2146.
        ...                      7.8....9.
        ...                      .........
        ...                      .5....3.1
        ...                      .2491...7
        ...                      9.....5..
        ...                      3...84...''')))
        632845179
        471369285
        895721463
        748153692
        163492758
        259678341
        524916837
        986237514
        317584926

    """
    def __init__(self, puzzle=None, n=9, m=None, random=False):
        if puzzle:
            puzzle = puzzle.split()
            self.n = n = len(puzzle)
            initial = self._encode_puzzle(puzzle)
        else:
            self.n = n
            initial = ()
        if m is None:
            m = int(ceil(sqrt(n)))
        assert(0 < n <= len(DIGITS))
        assert(n % m == 0)
        def constraints(choice):
            d, x, y = self._decode_choice(choice)
            block = m * (x // m) + y // (n // m)
            return [a + 4 * (b + n * c) for a, b, c in [
                (0, x, y),     # Any digit at x, y.
                (1, d, y),     # Digit d in row y.
                (2, d, x),     # Digit d in column x.
                (3, d, block), # Digit d in block.
            ]]
        self.solver = ExactCover({i: constraints(i) for i in range(n ** 3)},
                                 initial, random)

    def __next__(self):
        return self._decode_solution(next(self.solver))

    next = __next__             # for compatibility with Python 2

    def _encode_choice(self, d, x, y):
        """Encode the placement of d at (x, y) as a choice."""
        n = self.n
        assert(0 <= d < n and 0 <= x < n and 0 <= y < n)
        return d + n * (x + n * y)

    def _decode_choice(self, choice):
        """Decode a choice into a (digit, x, y) tuple."""
        choice, digit = divmod(choice, self.n)
        y, x = divmod(choice, self.n)
        return digit, x, y

    def _encode_puzzle(self, puzzle):
        """Encode a Sudoku puzzle and yield initial choices."""
        for y, row in enumerate(puzzle):
            for x, d in enumerate(row):
                digit = DIGITS.find(d)
                if digit != -1:
                    yield self._encode_choice(digit, x, y)

    def _decode_solution(self, solution):
        """Decode a Sudoku solution and return it as a string."""
        grid = make_grid(self.n)
        for d, x, y in map(self._decode_choice, solution):
            grid[y][x] = d
        return grid_to_puzzle(grid)


class ImpossiblePuzzle(Exception): pass


class AmbiguousPuzzle(Exception): pass


def solve(puzzle, m=None):
    """Solve the Sudoku puzzle and return its unique solution. If the
    puzzle is impossible, raise ImpossiblePuzzle. If the puzzle has
    multiple solutions, raise AmbiguousPuzzle.

    Optional argument m is the width of the blocks (default: the
    square root of n, rounded up).

    For example:

        >>> print(solve('''.6.5..9..
        ...                ..4.....6
        ...                .29..3..8
        ...                ....32..4
        ...                ..61.75..
        ...                1..95....
        ...                6..4..27.
        ...                2.....4..
        ...                ..7..6.8.'''))
        768514923
        314829756
        529763148
        975632814
        836147592
        142958637
        693481275
        281375469
        457296381
        >>> print(solve('''...8.....
        ...                ..1.....5
        ...                8....1...
        ...                7.8....9.
        ...                ...1.....
        ...                .5....3.1
        ...                ....1...7
        ...                9.....5..
        ...                3........'''))
        ... # doctest: +IGNORE_EXCEPTION_DETAIL
        Traceback (most recent call last):
          ...
        ImpossiblePuzzle: no solutions

    """
    solutions = list(itertools.islice(Sudoku(puzzle, m=m), 2))
    if len(solutions) == 1:
        return solutions[0]
    elif len(solutions) == 0:
        raise ImpossiblePuzzle('no solutions')
    else:
        raise AmbiguousPuzzle('two or more solutions')

def make_puzzle(n=9, m=None):
    """Return a random nxn Sudoku puzzle.
    
    The puzzle returned is minimal in the sense that no
    symmetric pair of givens can be removed without making the puzzle
    ambiguous.

    The optional arguments are n, the size of the puzzle (default: 9)
    and m, the width of the blocks (default: the square root of n,
    rounded up).

    """
    grid = puzzle_to_grid(next(Sudoku(n=n, m=m, random=True)))
    coords = list(divmod(i, n) for i in range(n ** 2))
    random.shuffle(coords)
    for i, j in coords:
        g = deepcopy(grid)
        g[i][j] = -1
        try:
            solve(grid_to_puzzle(g))
            grid = g
        except AmbiguousPuzzle:
            pass
    return grid_to_puzzle(grid)

In [4]:
for puzzle in map(make_puzzle, (4, 9, 12)):
    print('\n'.join('{}    {}'.format(*a) for a in zip(puzzle.split(), solve(puzzle).split())), '\n')

Sudoku:
[0, 0, 7, 0, 6, 2, 0, 3, 0]
[6, 2, 3, 0, 0, 0, 0, 7, 0]
[0, 1, 0, 4, 0, 7, 0, 2, 0]
[0, 0, 0, 6, 0, 0, 0, 0, 0]
[1, 0, 8, 0, 9, 0, 5, 0, 7]
[2, 0, 6, 0, 0, 5, 0, 1, 0]
[0, 0, 2, 0, 4, 0, 0, 5, 0]
[0, 0, 0, 0, 2, 8, 7, 9, 6]
[0, 0, 0, 0, 7, 1, 3, 0, 2]
Solved:
[4, 8, 7, 1, 6, 2, 9, 3, 5]
[6, 2, 3, 8, 5, 9, 1, 7, 4]
[9, 1, 5, 4, 3, 7, 6, 2, 8]
[7, 5, 4, 6, 1, 3, 2, 8, 9]
[1, 3, 8, 2, 9, 4, 5, 6, 7]
[2, 9, 6, 7, 8, 5, 4, 1, 3]
[3, 7, 2, 9, 4, 6, 8, 5, 1]
[5, 4, 1, 3, 2, 8, 7, 9, 6]
[8, 6, 9, 5, 7, 1, 3, 4, 2]


In [ ]:
test_puzzle = make_puzzle(9)
print(repr(np.array(puzzle_to_grid(test_puzzle)) + 1), '\n')
print(repr(np.array(puzzle_to_grid(solve(test_puzzle))) + 1))